In [1]:
import os
import numpy as np
from dataclasses import dataclass, fields
from VASP_job.code.main import VASP_job

import pandas as pd
import ase
from ase.spacegroup import crystal

from pyiron.atomistics.structure.atoms import ase_to_pyiron
from pyiron.atomistics.structure.atoms import pyiron_to_ase

import matplotlib.pyplot as plt


# Define a custom formatting function for vectors in pandas
format_vector = lambda vector: '[' + ', '.join(['{:.10f}'.format(x) for x in vector]) + ']'

In [2]:
a = 3.95
alpha = 90
structure = crystal(('Ni', 'Mn', 'N'),
                       basis=[(0.00, 0.00, 0.00), (0.00, 0.50, 0.50), (0.50, 0.50, 0.50)],
                       spacegroup=221,
                       cellpar=[a, a, a, alpha, alpha, alpha])
structure.symbols.species()

{'Mn', 'N', 'Ni'}

In [3]:
Dtheta = 120 * np.pi/180
theta = -Dtheta

number_of_atoms = len(structure)
magdirs   = []
ms        = []
B_CONSTR  = []
for i in range(number_of_atoms):
   if 'Mn' == structure.get_chemical_symbols()[i]:
      theta += Dtheta
      magdirs.append([np.cos(theta), np.sin(theta), 0])
      ms.append(0.5)
      B_CONSTR.append([0, 0, 0])
   else:
      magdirs.append([0, 0, 0])
      ms.append(1)
      B_CONSTR.append([0, 0, 0])

In [4]:
structure.new_array("magdirs", magdirs, dtype=float)
structure.new_array("ms", ms, dtype=float)
structure.new_array("B_CONSTR", B_CONSTR, dtype=float)

In [5]:
structure = structure.repeat((2,1,1))
# structure = ase_to_pyiron(structure)
# structure.set_repeat([2,2,2])
# structure = pyiron_to_ase(structure)

In [6]:
# Create a defition that does all this?

executable_path= "/home/emendive/workbench/work/from_claix/codes/VASP/vasp.5.4.4-flag4/bin"
potential_path = "/home/emendive/workbench/work/from_claix/codes/VASP/potentials/potpaw_PBE"
vasp = VASP_job(verbose="low", executable_path=executable_path, potential_path=potential_path)

In [7]:
vasp.prepare_bfields()
vasp.set_calculation(structure)

In [8]:
vasp.restart_from_charge(kpoints="3 2 2", LAMBDA=3)

In [9]:
this_dir = os.getcwd()
vasp.restart_from_charge(cwd_new=this_dir+"/aaa", kpoints="3 3 2", LAMBDA=5)